In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
2,How contributions support WHO's work in ongoin...,https://reliefweb.int/report/world/how-contrib...,"Thu, 21 Jan 2021 18:37:00 GMT",2021-01-21 18:37:00+00:00
0,Fast-spreading COVID variant can elude immune ...,https://www.nature.com/articles/d41586-021-001...,"Thu, 21 Jan 2021 17:37:36 GMT",2021-01-21 17:37:36+00:00
5,Memphis Grizzlies' next three games postponed ...,https://www.espn.com/nba/story/_/id/30753440/m...,"Thu, 21 Jan 2021 16:45:25 GMT",2021-01-21 16:45:25+00:00
1,Eli Lilly: Antibody drug prevents COVID-19 in ...,https://www.modernhealthcare.com/post-acute-ca...,"Thu, 21 Jan 2021 16:37:42 GMT",2021-01-21 16:37:42+00:00
9,'We need more': Cities and hospitals feel pinc...,https://www.nbcnews.com/news/us-news/we-need-m...,"Thu, 21 Jan 2021 15:08:00 GMT",2021-01-21 15:08:00+00:00
8,"Ohio provider suspended for mishandling, spoil...",https://www.nbcnews.com/news/us-news/ohio-prov...,"Thu, 21 Jan 2021 14:54:00 GMT",2021-01-21 14:54:00+00:00
3,"On Day 2, Biden To Focus On COVID-19 Strategy ...",https://www.wabe.org/on-day-2-biden-to-focus-o...,"Thu, 21 Jan 2021 13:28:23 GMT",2021-01-21 13:28:23+00:00
7,New CDC director says Covid vaccine won't be i...,https://www.nbcnews.com/news/us-news/new-cdc-d...,"Thu, 21 Jan 2021 13:28:00 GMT",2021-01-21 13:28:00+00:00
6,Covid: How a £20 gadget could save lives BBC ...,https://www.bbc.com/news/health-55733527,"Thu, 21 Jan 2021 13:11:00 GMT",2021-01-21 13:11:00+00:00
4,"COVID Can Cause Forgetfulness, Psychosis, Mani...",https://www.scientificamerican.com/article/cov...,"Thu, 21 Jan 2021 11:48:45 GMT",2021-01-21 11:48:45+00:00
